# Проект: Обучение с учителем: качество модели

## Описание проекта

Интернет-магазин «В один клик» продаёт разные товары: для детей, для дома, мелкую бытовую технику, косметику и даже продукты. Отчёт магазина за прошлый период показал, что активность покупателей начала снижаться. Привлекать новых клиентов уже не так эффективно: о магазине и так знает большая часть целевой аудитории. Возможный выход — удерживать активность постоянных клиентов. Сделать это можно с помощью персонализированных предложений.

«В один клик» — современная компания, поэтому её руководство не хочет принимать решения просто так — только на основе анализа данных и бизнес-моделирования. У компании есть небольшой отдел цифровых технологий, и вам предстоит побыть в роли стажёра в этом отделе.

Итак, вашему отделу поручили разработать решение, которое позволит персонализировать предложения постоянным клиентам, чтобы увеличить их покупательскую активность.

## Как решать задачу


1. Нужно промаркировать уровень финансовой активности постоянных покупателей. В компании принято выделять два уровня активности: «снизилась», если клиент стал покупать меньше товаров, и «прежний уровень».

2. Нужно собрать данные по клиентам по следующим группам:
- Признаки, которые описывают коммуникацию сотрудников компании с клиентом.
- Признаки, которые описывают продуктовое поведение покупателя. Например, какие товары покупает и как часто.
- Признаки, которые описывают покупательское поведение клиента. Например, сколько тратил в магазине.
- Признаки, которые описывают поведение покупателя на сайте. Например, как много страниц просматривает и сколько времени проводит на сайте.

**Основные этапы работы:**

1. Загрузка данных и изучение общей информации о датафреймах
2. Предобработка данных
 - изучение и обработка пропущенных значений
 - проверка соответствия типов данных
 - поиск и обработка убликатов
3. Исследовательский анализ данных
4. Объединение датафреймов и корреляционный анализ признаков итоговой таблицы
5. Корреляционный анализ
6. Обучение моделей испульзуя пайплайны
7. Анализ важности признаков с помощью **SHAP**
8. Сегментация покупателей
9. Общий вывод

**Установка и импорт нужных библиотек**

In [1]:
! pip install -Uq matplotlib -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 19.9 MB/s eta 0:00:00


In [2]:
!pip install -U scikit-learn -q # -q убирает необязательные выводы в командах Linux

In [3]:
!pip install shap -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.1/540.1 kB 18.6 MB/s eta 0:00:00


In [4]:
!pip install phik -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.1/686.1 kB 12.2 MB/s eta 0:00:00


In [5]:
!pip install mlxtend -q

In [6]:
import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from phik import phik_matrix
from phik.report import plot_correlation_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.dummy import DummyClassifier
from sklearn.inspection import permutation_importance
from sklearn.pipeline import Pipeline

from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler,
    MinMaxScaler,
    RobustScaler
)

from sklearn.metrics import (
    f1_score,
    roc_auc_score,
    accuracy_score,
    precision_score,
    recall_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    classification_report
)

from mlxtend.plotting import plot_decision_regions

In [7]:
df_market_file = pd.read_csv('/datasets/market_file.csv', sep=',', decimal='.')
df_market_money = pd.read_csv('/datasets/market_money.csv', sep=',', decimal='.')
df_market_time = pd.read_csv('/datasets/market_time.csv', sep=',', decimal='.')
df_money = pd.read_csv('/datasets/money.csv', sep=';', decimal=',')

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/market_file.csv'

In [ ]:
df_market_file.head(10)

In [ ]:
df_market_file.shape

In [ ]:
df_market_file.info()

In [ ]:
df_market_file.describe().T

In [ ]:
df_market_money.head(10)

In [ ]:
df_market_money.shape

In [ ]:
df_market_money.info()

In [ ]:
df_market_money.describe().T

In [ ]:
df_market_time.head(10)

In [ ]:
df_market_time.shape

In [ ]:
df_market_time.info()

In [ ]:
df_market_time.describe().T

In [ ]:
df_money.head(10)

In [ ]:
df_money.shape

In [ ]:
df_money.info()

In [ ]:
df_money.describe().T

Данные соответствуют описанию. Судя по количеству строк, у нас 1300 уникальных клиентов

Нужно привести названия столбцов к одному виду к одному виду

**Предобработка данных**

Зададим датафреймам имена, чтобы удобно выводить их в циклах

In [ ]:
df_market_file.name = 'df_market_file'
df_market_money.name = 'df_market_money'
df_market_time.name = 'df_market_time'
df_money.name = 'df_money'

Будем перебирать сразу все датафреймы в циклах

In [ ]:
df_list = [df_market_file, df_market_money, df_market_time, df_money]

**Изменим названия столбцов**

In [ ]:
for df in df_list:
    df.columns = [x.lower() for x in df.columns]
    df.columns = [x.replace(' ', '_') for x in df.columns]

In [ ]:
df_list

**Поиск пропущенных значений**

In [ ]:
for df in df_list:
    print(f'Пропущенные значения {df.name}:\n{df.isna().sum()}')
    print('\n')

Пропусков нет

Все данные соотвествуют своему типу, это можно заметить на этапе открытия и обзора датафреймов

**Поиск явных дубликатов**

In [ ]:
for df in df_list:
    print(f'Явных дубликатов в "{df.name}": {df.duplicated().sum()}')

Явных дубликатов нет

**Поиск неявных дубликатов**

Изучим уникальные значения столбцов в каждом датафрейме

**Изучим `df_market_file`**

In [ ]:
for col in df_market_file[df_market_file.select_dtypes(exclude='number').columns]:
    print(f'Уникальные значения столбца "{col}":\n')
    print(f'{df_market_file[col].value_counts()}\n')
    print(f'Количество уникальных значений столбца "{col}": {len(df_market_file[col].unique())}')
    print('\n')

print('\nКоличество уникальных значений столбца "id":', len(df_market_file['id'].unique()))

Можем заметить, что в столбце `тип_сервиса` есть значения: `стандарт`, `стандартт`. Это ошибка

In [ ]:
df_market_file['тип_сервиса'] = df_market_file['тип_сервиса'].str.replace('стандартт', 'стандарт')

In [ ]:
print('Столбец "тип_сервиса":\n', df_market_file['тип_сервиса'].value_counts())

Теперь всё нормально

**Изучим `df_market_money`**

In [ ]:
for col in df_market_money[df_market_money.select_dtypes(exclude='number').columns]:
    print(f'Уникальные значения столбца "{col}":\n')
    print(f'{df_market_money[col].value_counts()}\n')
    print(f'Количество уникальных значений столбца "{col}": {len(df_market_money[col].unique())}')
    print('\n')

print('\nКоличество уникальных значений столбца "id":', len(df_market_money['id'].unique()))

Неявных дубликатов нет

**Изучим `df_market_time`**

In [ ]:
for col in df_market_time[df_market_time.select_dtypes(exclude='number').columns]:
    print(f'Уникальные значения столбца "{col}":\n')
    print(f'{df_market_time[col].value_counts()}\n')
    print(f'Количество уникальных значений столбца "{col}": {len(df_market_time[col].unique())}')
    print('\n')

print('\nКоличество уникальных значений столбца "id":', len(df_market_time['id'].unique()))

Неявных дубликатов нет

**В `df_money` неявных дубликатов не может быть**

**Вывод по этому этапу:**

Пропусков не было, явных дубликатов тоже.

Там, где были неявные дубликаты, мы исправили.

**Исследовательский анализ**

Изучим числовые столбцы `df_market_file`

In [ ]:
df_market_file.drop('id', axis=1).hist(figsize=(15, 12), ec='black', alpha=0.7, bins=30)
plt.suptitle('Распределения данных для числовых столбцов "df_market_file"', size=15)

Все распределения похожи на нормальные, но в столбце `акционные_покупки` можно наблюдить два "горба", то есть распределение бимодальное

Напишем функции для более удобного и быстрого анализа данных

In [ ]:
def num_func(data):
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    plt.subplot(1,2,1)
    plt.hist(data, bins=30, ec='black', color='y', alpha=0.5)
    plt.title('Распределение', size=15)
    plt.ylabel('Частота', size=12)
    plt.grid(linestyle='dashed')
    plt.subplot(1,2,2)
    plt.boxplot(data, vert=False)
    plt.title('Ящик с усами', size=15)
    plt.grid(linestyle='dashed')
    plt.show()
    print('Описательная статистика:')
    print(data.describe())

In [ ]:
colors = sns.color_palette("husl", 9)
def cat_func(data):
    print()
    print(data.value_counts())
    data.value_counts(ascending=True).plot(kind='barh',
                             alpha=0.5)
    plt.title('Столбчатая диаграмма', size=15)
    plt.xticks(rotation=0)
    plt.xlabel('Количество', size=12)
    plt.ylabel('Значения', size=12)
    plt.show()
    print('\n')
    data.value_counts().plot(kind='pie',
                             legend=True,
                             autopct='%.2f%%',
                             textprops={'color':'white', 'size':15},
                             figsize=(8, 6), colors=colors)
    plt.legend(bbox_to_anchor=(1, 1), prop={'size': 15})
    plt.title('Круговая диаграмма', size=15)
    plt.ylabel(None)
    plt.show()
    print('\n')

Изучим числовые столбцы `df_market_money`

In [ ]:
df_market_file_num = df_market_file[df_market_file.select_dtypes(include='number').columns].drop(['id'], axis=1)
for i in df_market_file_num:
    print(f'Столбец "{i}"')
    num_func(df_market_file_num[i])
    plt.show()
    print('\n')

Изучим категориальные признаки `df_market_money`

In [ ]:
df_market_file_cat = df_market_file[df_market_file.select_dtypes(exclude='number').columns]
for i in df_market_file_cat:
    print(f'Столбец "{i}"')
    cat_func(df_market_file_cat[i])
    plt.show()

**Вывод по датафрейму `df_market_file`**

`маркет_актив_6_мес` - близится к нормальному распределению

`маркет_актив_тек_мес` - принимает всего 3 значения

`длительность` - близится к нормальному распределению

`акционные_покупки` - бимодальное распределение, есть пользователи, которые почти всегда покупают только товары по акции

`средний_просмотр_категорий_за_визит` - нормальное распределение

`неоплаченные_продукты_штук_квартал` - нормальное распределение со смещением

`ошибка_сервиса` - нормальное распределение

`страниц_за_визит` - нормальное распределение

По категориальным признакам:

Большая часть клиентов осталась на том же уровне активности и на стандартном типе сервиса. **Самая популярная категория** - товары для детей, меньше всего покупают кухонную посуду

**Изучим `df_market money`**

In [ ]:
df_market_money_num = (df_market_money[df_market_money.select_dtypes(include='number').columns].drop(['id'], axis=1))
for i in df_market_money_num:
    print(f'Столбец "{i}"')
    num_func(df_market_money_num[i])
    plt.show()
    print('\n')

In [ ]:
df_market_money.drop('id', axis=1).hist(figsize=(7, 5), ec='black', alpha=0.7, color='yellow', bins=30)
plt.title('Распределения данных для "выручка"', size=15)
plt.axvline(df_market_money['выручка'].median(), color='g', linestyle='dashed', linewidth=2)
plt.axvline(df_market_money['выручка'].mean(), color='r', linestyle='-', linewidth=2)
plt.legend(['Медиана', 'Среднее', 'Выручка'], prop={'size': 12})
plt.xlabel('Выручка', size=12)
plt.ylabel('Частота', size=12)

На этой диаграмме мы ничего не можем увидеть, судя по тому, что основной график в левой стороне, это из-за большого максимального значения(выброса)

In [ ]:
df_market_money.boxplot(column=['выручка'], figsize=(15,5), vert=False)
plt.title('Диаграмма размаха для выручки', size=15);

Мы видим два аномальных значения: 0 и больше 100000.

In [ ]:
df_market_money.describe().T

In [ ]:
(df_market_money
 .query('0 < выручка < 106862.0')
 .drop('id', axis=1)
 .hist(
     figsize=(7, 5),
     ec='black',
     alpha=0.8,
     color='yellow',
     bins=30,
     legend=True)
)
plt.title('Распределения данных для "выручка"', size=15, y=1.05)
plt.axvline(df_market_money['выручка'].median(), color='g', linestyle='dashed', linewidth=2)
plt.axvline(df_market_money['выручка'].mean(), color='r', linestyle='-', linewidth=2)
plt.legend(['Медиана', 'Среднее', 'Выручка'], prop={'size': 12})
plt.xlabel('Выручка', size=12)
plt.ylabel('Частота', size=12)

Наблюдаем нормальное распределение значений.

**Категориальные признаки**

In [ ]:
df_market_money_cat = df_market_money[df_market_money.select_dtypes(exclude='number').columns]
for i in df_market_money_cat:
    print(f'Столбец "{i}"')
    cat_func(df_market_money_cat[i])
    plt.show()

**Вывод по `df_market_money`**

`Выручка` - нормальное распределение, но есть выбросы

Категориальные признаки - распределение выручки по периодам равномерное

**Изучим `df_market_time`**

In [ ]:
df_market_time_num = df_market_time[df_market_time.select_dtypes(include='number').columns].drop(['id'], axis=1)
for i in df_market_time_num:
    print(f'Столбец "{i}"')
    num_func(df_market_time_num[i])
    plt.show()
    print('\n')

**Категориальные признаки**

In [ ]:
df_market_time_cat = df_market_time[df_market_time.select_dtypes(exclude='number').columns]
for i in df_market_time_cat:
    print(f'Столбец "{i}"')
    cat_func(df_market_time_cat[i])
    plt.show()

**Вывод по `df_market_time`**

`минут` - нормальное распределение

Категориальные признаки - пользователи проводят одинаковое количество времени в обоих периодах

**Изучим `df_money`**

In [ ]:
df_money_num = df_money.drop(['id'], axis=1)
for i in df_money_num:
    print(f'Столбец "{i}"')
    num_func(df_money_num[i])
    plt.show()
    print('\n')

**Категориальных признаков нет**

**Вывод по `df_money`**

`прибыль` - нормальное распределение

**Отберём клиентов с покупательской активностью не менее трёх месяцев**

In [ ]:
no_active_id = df_market_money.loc[(df_market_money['выручка'] == 0)]['id'].unique()
active_users_three_months = df_market_money[df_market_money.id.isin(no_active_id) == False]
print('Пользователей с постоянной активностью:',active_users_three_months['id'].nunique())

In [ ]:
df_market_money = df_market_money[df_market_money.id.isin(no_active_id) == False]
df_market_money['id'].nunique()

**Клиенты с активностью менее трёх месяцев отсеяны**

Теперь можем разобраться с выбросом в столбце `выручка` из датафрейма `df_market_money`

In [ ]:
df_market_money.describe().T

Так как мы отобрали активных пользователей, то значения 0 там нет, а вот значение **106862.2** слишком сильно выбивается, это будет сказываться на обучении моделей в дальнейшем, поэтому его можно удалить

In [ ]:
id_to_delete = df_market_money[df_market_money['выручка'] == 106862.2]['id'].tolist()
df_market_money = df_market_money.query('id != @id_to_delete')

**Общий вывод по исследовательскому анализу**

- данные во всех датафреймах визуализированы и проанализированы
- удалили явный выброс, который помешал бы работе
- отобрали активных пользователей

**Объединение таблиц**

Объединим датафреймы `df_market_file`, `df_market_money` и `df_market_time`.

Данные о выручке и времени на сайте находятся в одном столбце для всех периодов. В итоговой таблице сделаем отдельный столбец для каждого периода.

In [ ]:
df_market_money_cur_mon = (df_market_money
                           .query('период == "текущий_месяц"')
                           .drop(['период'], axis=1)
                           .rename(columns={'выручка': 'выручка_текущий_месяц'}))

df_market_money_pre_mon = (df_market_money
                           .query('период == "предыдущий_месяц"')
                           .drop(['период'], axis=1)
                           .rename(columns={'выручка': 'выручка_предыдущий_месяц'}))

df_market_money_prepre_mon = (df_market_money
                              .query('период == "препредыдущий_месяц"')
                              .drop(['период'], axis=1)
                              .rename(columns={'выручка': 'выручка_предпредыдущий_месяц'}))

df_market_time_cur_mon = (df_market_time
                          .query('период == "текущий_месяц"')
                          .drop(['период'], axis=1)
                          .rename(columns={'минут': 'минут_текущий_месяц'}))

df_market_time_pre_mon = (df_market_time
                          .query('период == "предыдцщий_месяц"')
                          .drop(['период'], axis=1)
                          .rename(columns={'минут': 'минут_предыдущий_месяц'}))

In [ ]:
df_merged = (df_market_file
             .merge(df_market_money_cur_mon, on='id')
             .merge(df_market_money_pre_mon, on='id')
             .merge(df_market_money_prepre_mon, on='id')
             .merge(df_market_time_cur_mon, on='id')
             .merge(df_market_time_pre_mon, on='id')
             .reset_index(drop=True))

In [ ]:
df_merged.shape

**Вывод**

Создали новые столбцы для каждого периода, объединили таблицы. Итоговая размерность таблицы - 1296 строк и 18 столбцов

**Корреляционный анализ**

In [ ]:
phik_overview = (
    df_merged
    .drop('id', axis=1)
    .phik_matrix(verbose=False)
)
plot_correlation_matrix(
    phik_overview.values,
    x_labels=phik_overview.columns,
    y_labels=phik_overview.index,
    vmin=0, vmax=1, color_map='Greens',
    title='Корреляция',
    fontsize_factor=1.5,
    figsize=(20, 15)
)

In [ ]:
df_merged.info()

Все связи видны на матрице корреляций, стоит отметить высокую связь между признаками `выручка_предыдущий_месяц` и `выручка_текущий_месяц`. Мультиколлинеарности нет, потому что коэффициенты корреляции не достаточно высокие

**Использование пайплайнов**

Целевой признак - `покупательская активность`. Мы сделали предобработку данных, но надо проверить его на дисбаланс классов.

In [ ]:
print(round(df_merged['покупательская_активность'].value_counts(normalize=True), 2))

In [ ]:
df_merged['покупательская_активность'].value_counts(normalize=True).plot(kind='bar')
plt.title('Распределение классов целевого признака', size=14)
plt.xlabel('Показатель покупательской активности', size=12)
plt.ylabel('Частота встречаемости', size=12)
plt.xticks(rotation=0)

Наблюдается дисбаланс классов, поэтому будем выполнять стратификацию по целевому признаку

Стоит поменять названия классов на 1 и 0. Так как нам надо увеличивать покупательскую активность, поэтому надо искать людей, у которых она снизилась, поэтому за 1 возьмём именно таких клиентов

In [ ]:
cat_value_one = 'Снизилась'
cat_value_zero = 'Прежний уровень'
def to_binary_func(value):
    if value == cat_value_one:
        return 1
    elif value == cat_value_zero:
        return 0

df_merged['покупательская_активность'] = df_merged['покупательская_активность'].apply(to_binary_func)

In [ ]:
print(df_merged['покупательская_активность'].value_counts())

У нас задача бинарной классификации, поэтому будем использовать метрику `ROC-AUC`

In [ ]:
RANDOM_STATE = 42
TEST_SIZE = 0.25

X = df_merged.drop(columns=['покупательская_активность', 'id'])
y = df_merged['покупательская_активность']
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    random_state=RANDOM_STATE,
    test_size=TEST_SIZE,
    stratify=y
)


Нам нужно найти лучшую модель с лучшим набором гиперпараметров. Будем выбирать из этого списка:
- **`KNeighborsClassifier()`**:
 - **n_neighbors** от 2 до 4
- **`DecisionTreeClassifier()`**:
 - **max_depth** от 2 до 4
 - **max_features** от 2 до 4 включительно (у нас много признаков)
- **`LogisticRegression()`**:
 - **C** от 1 до 4
- **`SVC()`**:
 - **C** от 1 до 9

In [ ]:
ohe_columns = ['популярная_категория']
ord_columns = ['тип_сервиса', 'разрешить_сообщать']
num_columns = ['маркет_актив_6_мес',
               'маркет_актив_тек_мес',
               'длительность',
               'акционные_покупки',
               'средний_просмотр_категорий_за_визит',
               'неоплаченные_продукты_штук_квартал',
               'ошибка_сервиса',
               'страниц_за_визит',
               'выручка_текущий_месяц',
               'выручка_предыдущий_месяц',
               'выручка_предпредыдущий_месяц',
               'минут_текущий_месяц',
               'минут_предыдущий_месяц']
ohe_pipe = Pipeline(
    [('ohe', OneHotEncoder(drop='first', handle_unknown='error', sparse_output=False))]
)

ord_pipe = Pipeline(
    [('ord', OrdinalEncoder(
    categories=[
        ['стандарт', 'премиум'],
        ['да', 'нет']
    ],
    handle_unknown='use_encoded_value',
    unknown_value=np.nan))]
)
data_preprocessor = ColumnTransformer(
    [('ohe', ohe_pipe, ohe_columns),
     ('ord', ord_pipe, ord_columns),
     ('num', MinMaxScaler(), num_columns)],
    remainder='passthrough'
)
pipe_final = Pipeline(
    [
        ('preprocessor', data_preprocessor),
        ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))
    ]
)
param_grid = [
    {
        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE)],
        'models__max_depth': range(2, 6),
        'models__max_features': range(2, 6),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), RobustScaler()]
    },

    {
        'models': [KNeighborsClassifier()],
        'models__n_neighbors': range(2, 6),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), RobustScaler()]
    },

    {
        'models': [LogisticRegression(
            random_state=RANDOM_STATE,
            solver='liblinear',
            penalty='l1'
        )],
        'models__C': range(1, 5),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), RobustScaler()]
    },
    {
        'models': [SVC(
            random_state=RANDOM_STATE,
            kernel='linear',
             probability= True
        )],
        'models__C': range(1, 10),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), RobustScaler()]
    },
]
randomized_search = RandomizedSearchCV(
    pipe_final,
    param_grid,
    cv=5,
    scoring='roc_auc',
    random_state=RANDOM_STATE,
    n_jobs=-1
)

randomized_search.fit(X_train, y_train)
proba = randomized_search.predict_proba(X_test)
proba_one = proba[:, 1]
print('Лучшая модель и её параметры:\n\n', randomized_search.best_estimator_)
print('\n')
print('Параметры лучшей модели:\n',randomized_search.best_params_)
print('\n')
print ('Метрика лучшей модели на тренировочной выборке:', randomized_search.best_score_)
print('\n')
print(f'Метрика ROC-AUC на тестовой выборке: {round(roc_auc_score(y_test, proba_one), 3)}')

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
pd.DataFrame(randomized_search.cv_results_)[
    ['rank_test_score', 'param_models', 'mean_test_score','params']
].sort_values('rank_test_score')

Лучшая модель - **`SVC(kernel='linear')`** с `C` = 6 и скелером `MinMaxScaler()`

Метрика лучшей модели на тренировочной выборке: **0.8926089395180303**

Метрика ROC-AUC на тестовой выборке: **0.918**

**Анализ важности признаков**

Оценим важность признаков для лучшей модели и построим график важности с помощью метода **`SHAP`**.

In [ ]:
ohe_encoder = OneHotEncoder(sparse_output=False, drop='first')
X_train_ohe = ohe_encoder.fit_transform(X_train[ohe_columns])
X_test_ohe = ohe_encoder.transform(X_test[ohe_columns])
ohe_encoder_col_names = ohe_encoder.get_feature_names_out()

ord_encoder = OrdinalEncoder()
X_train_ord = ord_encoder.fit_transform(X_train[ord_columns])
X_test_ord = ord_encoder.transform(X_test[ord_columns])
ord_encoder_col_names = ord_encoder.get_feature_names_out()

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train[num_columns])
X_test_scaled = scaler.transform(X_test[num_columns])

X_train_ohe = pd.DataFrame(X_train_ohe, columns=ohe_encoder_col_names)
X_test_ohe = pd.DataFrame(X_test_ohe, columns=ohe_encoder_col_names)

X_train_ord = pd.DataFrame(X_train_ord, columns=ord_encoder_col_names)
X_test_ord = pd.DataFrame(X_test_ord, columns=ord_encoder_col_names)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=num_columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=num_columns)

X_train_first = pd.concat([X_train_ohe, X_train_ord], axis=1)
X_train_final = pd.concat([X_train_first, X_train_scaled], axis=1)

X_test_first = pd.concat([X_test_ohe, X_test_ord], axis=1)
X_test_final = pd.concat([X_test_first, X_test_scaled], axis=1)

In [ ]:
clf = SVC(kernel='linear', C=6, probability=True, random_state=42)
clf.fit(X_train_final, y_train)
y_pred = clf.predict(X_test_final)

clf_probas = clf.predict_proba(X_test_final)[:,1]

roc_auc_cv = cross_val_score(clf, X_train_final, y_train, scoring='roc_auc').mean()
print(f'ROC-AUC на тренировочной выборке: {round(roc_auc_cv, 3)}')
print(f'ROC-AUC на тестовой выборке: {round(roc_auc_score(y_test, y_pred), 3)}')


clf_f1 = f1_score(y_test, y_pred, pos_label=1)
print('F1-score:', round(clf_f1, 3))

In [ ]:
ConfusionMatrixDisplay.from_estimator(clf, X_test_final, y_test)
plt.title('Матрица ошибок', size=15)
plt.xlabel('Предсказанный класс', size=12)
plt.ylabel('Истинный класс', size=12)
plt.show()

Модель в 23 случаях совершает ошибку второго рода, значит мы можем упустить клиентов к которых снижается активность

In [ ]:
explainer = shap.LinearExplainer(clf, X_test_final)
shap_values = explainer(X_test_final)
sns.set_style('white')
shap.plots.bar(shap_values, max_display=20, show=False)
plt.title('График общей значимости признаков', size=15)
plt.show()

In [ ]:
shap.summary_plot(shap_values, X_test_final, show=False, plot_size=[12, 6], cmap='coolwarm')
plt.title('Визуализация вклада признаков в предсказания модели', size=15, y=1.03)
plt.show()

**Вывод по `SHAP` анализу**

5 самых влиятельных признаков:
- страниц_за_визит
- акционные_покупки
- минут_предыдущий_месяц
- средний_просмотр_категорий_за_визит
- минут_текущий_месяц

Исходя из визуализации вклада признаков:
- высокие значения `страниц_за_визит`, `минут_предыдущий_месяц` и `минут_текущий_месяц` уменьшают значения SHAP-объектов и увеличивают вероятность принадлежности наблюдений к классу 0
- высокие значения `акционные_покупки` увеличивают значения SHAP-объектов

**Изучем неверной классификации объектов**

In [ ]:
X_test_final['y_test'] = y_test.tolist()
X_test_final['y_pred'] = y_pred.tolist()
X_test_final['predict_proba'] = clf_probas

display(X_test_final[(X_test_final['y_test']==1)
                     & (X_test_final['y_pred']==0)
                     & (X_test_final['predict_proba']<=0.5)][['y_test',
                                                            'y_pred',
                                                            'predict_proba',
                                                            'страниц_за_визит',
                                                            'акционные_покупки']].sort_values(by='predict_proba'))

display(X_test_final[(X_test_final['y_test']==0)
                     & (X_test_final['y_pred']==1)
                     & (X_test_final['predict_proba']>0.5)][['y_test',
                                                            'y_pred',
                                                            'predict_proba',
                                                            'страниц_за_визит',
                                                            'акционные_покупки']].sort_values(ascending=False,
                                                                                              by='predict_proba'))

In [ ]:
indexes_one = X_test_final.index[(X_test_final['y_test']==1) & (X_test_final['y_pred']==0)].tolist()

for i in indexes_one:
    print('Index:', i)
    fig = plt.figure()
    shap.plots.waterfall(shap_values[i], show=False)
    plt.show()
    print('\n')

**Общие выводы по этому этапу:**
- Чем дольше клиент находится на сайте, тем выше вероятность того, что он что-то купит
- Акционные предложения оказывают большое влияние на активность клиентов

**Сегментация покупателей**

In [ ]:
def to_cat(value):
    if value == 1:
        return cat_value_one
    elif value == 0:
        return cat_value_zero

In [ ]:
clf_probas_train = clf.predict_proba(X_train_final)[:,1]
X_pred = clf.predict(X_train_final)

total_train = pd.concat([X_train, y_train], axis=1)
total_test = pd.concat([X_test, y_test], axis=1)

total_train['probas'] = clf_probas_train.tolist()
total_train['класс_прогноз'] = X_pred.tolist()
total_test['probas'] = clf_probas.tolist()
total_test['класс_прогноз'] = y_pred.tolist()

final_df = pd.concat([total_train, total_test])
final_df = pd.merge(final_df, df_merged['id'], left_index=True, right_index=True)

final_df['покупательская_активность'] = final_df['покупательская_активность'].apply(to_cat)
cat_value_one = 'Снизится'
cat_value_zero = 'Не изменится'
final_df = final_df.rename(columns={'класс_прогноз': 'прогноз_покуп_актив'})
final_df = final_df.merge(df_money, on='id')
final_df.head()

In [ ]:
num_func(final_df['прибыль'])

In [ ]:
num_func(final_df['probas'])

In [ ]:
def segment(data):
    if data > 0.2:
        return 'Выс. вероятность снижения'
    if data < 0.2:
        return 'Низ. вероятность снижения'

final_df['прогноз_покуп_активности'] = final_df['probas'].apply(segment)
selected_segment = final_df[(final_df['прибыль'] > 5) & (final_df['probas'] > 0.2)]
selected_segment_good = final_df[(final_df['прибыль'] > 5) & (final_df['probas'] < 0.2)]
print('Количество покупателей выбранного сегмента:', selected_segment['id'].count())
selected_segment = selected_segment.rename(columns={'разрешить_сообщать': 'рассылка'})
selected_segment.sort_values(by='прибыль', ascending=False)[['id',
                                                             'прибыль',
                                                             'probas',
                                                             'прогноз_покуп_активности',
                                                             'страниц_за_визит',
                                                             'акционные_покупки',
                                                             'популярная_категория',
                                                             'тип_сервиса',
                                                             'рассылка']]

In [ ]:
selected_segment[['probas', 'прибыль']].hist(bins=30, ec='black', alpha=0.5, legend=True, figsize=(15, 5), color='r')

- Видим, что у многих покупателей выбранного сегмента вероятность принадлежности к классу 1 (снижение) даже выше 0.9, что говорит об очень высокой вероятности снижения их покупательской активности вплоть до её сведения к нулю, то есть ухода к конкурентам
- Касаемо прибыльности можно сказать, что у большинства покупателей показатель прибыльности до 0.6

In [ ]:
print('Снижение покупательской активности (количество покупателей)')
print('Низкая вероятность:', selected_segment_good['id'].count())
print('Высокая вероятность:', selected_segment['id'].count())
print('\n')
(final_df[(final_df['прибыль'] > 5)]['прогноз_покуп_активности']
 .value_counts(ascending=True)
 .plot(kind='barh', color='r', ec='black', alpha=0.5))
plt.title('Покупатели с разной вероятностью\nснижения покупательской активности\n', size=15)
plt.xlabel('Количество покупателей ', size=12)
plt.show()
print('\n')
ax = selected_segment_good['прибыль'].plot(
    kind='hist',
    histtype='bar',
    bins=25,
    linewidth=1,
    label='raw',
    figsize=(10,7),
    ec='black',
    alpha=0.6,
    facecolor='y')

selected_segment['прибыль'].plot(
    kind='hist',
    histtype='bar',
    bins=25,
    linewidth=1,
    ax=ax,
    grid=False,
    figsize=(10, 7),
    ec='black',
    alpha=0.5,
    facecolor='r')
plt.grid(linestyle='dotted', color='dodgerblue')
plt.title('Распределение прибыльности для покупателей\nс разной вероятностью снижения покупательской активности\n', size=15)
plt.xlabel('Показатель прибыльности', size=12)
plt.ylabel('Количество покупателей', size=12)
plt.legend(['Низкая вероятность',
            'Высокая вероятность'], prop={'size': 13}, title='Снижение покупательской активности')
plt.show()

Учитывая соотношение покупателей с низкой и высокой вероятностью снижения покупательской активности и исходя из распределения прибыльности для данных покупателей по отдельности можно сказать, что прибыльность покупателей с высокой вероятностью снижения покупательской активности сопоставима с покупателями с низкой вероятностью.

То есть покупатели с высокой вероятность снижения покупательской активности с точки зрения прибыльности так же важны как и остальные. Мероприятия по их удержанию и повышению покупательской активности будут более чем оправданы, так как маржинальность их покупок такая же как у остальных покупателей, а в отдельных случаях даже выше.

In [ ]:
print('Разрешение на рекламные рассылки в выбранном сегменте:')
print(selected_segment['рассылка'].value_counts())

In [ ]:
print('Популярные категории в выбранном сегменте:')
print(selected_segment['популярная_категория'].value_counts())
print('\n')
print('Типы сервиса в выбранном сегменте:')
print(selected_segment['тип_сервиса'].value_counts())
print('\n')

selected_segment['страниц_за_визит'].hist(bins=25, color='r', ec='black', alpha=0.5, width=0.8)
plt.title('Распределение страниц за визит', size=15)
plt.xlabel('Страниц за визит', size=12)
plt.ylabel('Количество покупателей', size=12)
plt.show()
print('\n')

selected_segment['акционные_покупки'].hist(bins=25, color='r', ec='black', alpha=0.5)
plt.title('Распределение долей акционных покупок', size=15)
plt.xlabel('Доля', size=12)
plt.ylabel('Количество покупателей', size=12)
plt.show()

Есть вид покупателей, у которых доля акционных покупок выше 0.8

In [ ]:
print('Количество покупателей с долей акционных покупок выше 80%:',
      selected_segment[selected_segment['акционные_покупки'] > 0.8]['id'].count())
print('\n')

print('Их отношение к рассылкам:')
print(selected_segment[selected_segment['акционные_покупки'] > 0.8]['рассылка'].value_counts())
print('\n')

print('Самые популярные категории покупателей с долей акционных покупок выше 80%:')
print(selected_segment[selected_segment['акционные_покупки'] > 0.8]['популярная_категория'].value_counts())
print('\n')
print('Типы сервиса покупателей с долей акционных покупок выше 80%:')
print(selected_segment[selected_segment['акционные_покупки'] > 0.8]['тип_сервиса'].value_counts())
print('\n')

(selected_segment[selected_segment['акционные_покупки'] > 0.8]['популярная_категория']
 .value_counts(ascending=True)
 .plot(kind='barh', color='r', ec='black', alpha=0.5, width=0.7))
plt.title('Самые популярные категории покупателей\nс долей акционных покупок выше 80%\n', size=15)
plt.show()
print('\n')

**Общий вывод**:

Поставленная задача:

- Подготовить имеющиеся данные для обучения
- Разработать модель, которая предскажет вероятность снижения покупательской активности
- Выделить сегмент покупателей и разработать персонализированные предложения, чтобы увеличить их покупательскую активность

Исходные данные:

- `market_file.csv` - данные о поведении покупателя на сайте, о коммуникациях с покупателем и его продуктовом поведении
- `market_money.csv` - данные о выручке, которую получает магазин с покупателя, то есть сколько покупатель всего потратил за период взаимодействия с сайтом
- `market_time.csv` - данные о времени (в минутах), которое покупатель провёл на сайте в течение периода
- `money.csv` - данные о среднемесячной прибыли покупателя за последние 3 месяца: какую прибыль получает магазин от продаж каждому покупателю

Проведённая предобработка:

Датафреймы `df_market_file`, `df_market_money`, `df_market_time` и `df_money` были проверены:

- на наличие пропусков в данных
- на соответсвие данных своему типу
- на наличие явных и неявных дубликатов
В результате проверки:

- пропуски в данных обнаружены не были;
- типы данных во всех четырёх датафреймах корректны;
- явные дубликаты в имеющихся датафреймах отсутствуют;
- в датафрейме df_market_file были устранены неявные дубликаты в столбце тип_сервиса;
- в остальных датафреймах неявные дубликаты обнаружены не были.

Для поиска лучшей модели:

- был проведён корреляционный анализ всех признаков с целевым
- был подготовлен объединённый датафрейм для моделирования
- данные были разделены на выборки с учётом стратифакации, так как был обнаружен дисбаланс классов
- для выбора лучшей комбинации модели и гиперпараметров был создан пайплайн, в который вошли модели:
 - **`KNeighborsClassifier()`**:
   - **n_neighbors** от 2 до 4
 - **`DecisionTreeClassifier()`**:
   - **max_depth** от 2 до 4
   - **max_features** от 2 до 4 включительно (у нас много признаков)
 - **`LogisticRegression()`**:
   - **C** от 1 до 4
 - **`SVC()`**:
   - **C** от 1 до 9

эффективность выбранной модели оценивали метрикой ROC-AUC на тренировочной и тестовой выборках. Данная метрика лучше всего подходит для задачи бинарной классификации с несбалансированным целевым признаком.

В результате была выбрана лучшая модель:

Лучшая модель - **`SVC(kernel='linear')`** с `C` = 6 и скелером `MinMaxScaler()`

Метрика лучшей модели на тренировочной выборке: **0.8926089395180303**

Метрика ROC-AUC на тестовой выборке: **0.918**

Выбранный сегмент для анализа:

- покупатели с высокой прибыльностью (>5)
- и высокой вероятностью снижения покупательской активности (>0.2)

Предложения для выбранного сегмента покупателей:

- Предложить пользователям, склонным к акционным покупкам, промокоды на скидки на товары тех категорий, которые они предпочитают, исходя из данных за прошлые месяцы

- Попробовать затянуть покупателей сегмента глубже в сайт, сделать так, чтобы они просматривали больше страниц. Больше страниц больше контактов с сайтом, а значит и больше вероятность совершения покупки

- Пользователям со стандартным обслуживанием сделать выгодное предложение на переход на премиум-обслуживание

- Внутри сегмента предложить покупателям категорий Косметика и аксессуары и Товары для детей скидки на данные категории